### Import packages and set TEMP_FOLDER to save dictionary, corpus and trained model.

In [24]:
import os
import logging
import tempfile
import nltk
import sys
import time
from gensim import corpora, models, similarities
from six import iteritems
from nltk.corpus import stopwords
from nltk.tokenize import *
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
TEMP_FOLDER = tempfile.gettempdir()

In [25]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

### Define a function to create dictionary and corpus and train lsi model. 

In [29]:
def create_dict_and_corpus (q_text, a_text):
    #usage: "questions.txt", "answers.txt", "dictionary.dict", "corpus.mm"
    #Create dictionary
    nltk.extract_rels
    lemma = nltk.wordnet.WordNetLemmatizer()
    stoplist = set(stopwords.words('english'))
    #TAG NOT COMPLETE!!!!!
    tags=['WRB', 'WP', 'WP$', 'WPT', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'NNP', 'NNPS', 'NN', 'NNS', 'JJ', 'JJS', 'JJ']
    tokenizer = RegexpTokenizer(r'\w+')
    text_cleaned = []
    for q_line, a_line in zip(open(q_text), open(a_text)):
        line_cleaned = []
        tokens = tokenizer.tokenize(q_line.lower()+a_line.lower())
        pos = nltk.pos_tag(tokens)
        for i, j in pos:
            if (i not in stoplist):
                if (j in tags):
                    line_cleaned.append(lemma.lemmatize(i))
                    line_cleaned.append(i)
        text_cleaned.append(line_cleaned)
    dictionary=corpora.Dictionary(text_cleaned)
    dictionary.compactify()
    dictionary.save(os.path.join(TEMP_FOLDER, 'dictionary.dict'))
    #Create corpus
    class Corpus(object):
        def __iter__(self):
            for q_line, a_line in zip(open(q_text), open(a_text)):
                line = q_line.lower().split()+a_line.lower().split()
                yield dictionary.doc2bow(line)
    corpus = Corpus()
    corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.mm'), corpus)
    corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'corpus.mm'))
    #Train tf-idf model and lsi model
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]
    lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=5)
    lsi.save(os.path.join(TEMP_FOLDER,'model.lsi')) # 
    #Create similarities matrix
    corpus_lsi = lsi[corpus_tfidf]
    index = similarities.MatrixSimilarity(lsi[corpus])
    index.save(os.path.join(TEMP_FOLDER, 'index.index'))

### Define a function to process one user query.

In [56]:
def answer_to_user_query(user_query, dictionary, lsi, index, answer_lines):
    #user_query=input("How can I help you? ")
    lemma = nltk.wordnet.WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'\w+')
    #Process query
    tokens = [lemma.lemmatize(word) for word in tokenizer.tokenize(user_query)]
    vec_bow=dictionary.doc2bow(tokens)
    vec_lsi=lsi[vec_bow]
    sims = index[vec_lsi]
    sims = sorted(enumerate(sims), key=lambda item:-item[1])
    match=sims[0][0]
    answer = nltk.sent_tokenize(answer_lines[match])
    if (len(answer)<3):
        for line in answer:
            print(line)
            time.sleep(len(line)*0.04)
    else:
        print(answer[0]+'...')
        time.sleep(len(answer[0])*0.04)
        user_response=input("Is this answer relevant?(yes/no)")
        if (user_response=='yes'):
            time.sleep(2)
            print('Great! Here is the rest of the answer:')
            time.sleep(2)
            for i in range(1, len(answer)):
                print(answer[i])
                time.sleep(len(answer[i])*0.04)
        else:
            print('Sorry. You will need to ask someone else')  

### Generate and load the dictionary, corpus, model and index matrix once

In [34]:
#Execute only once
q_text='questions.txt'
a_text='answers.txt'
create_dict_and_corpus (q_text, a_text)

dictionary = corpora.Dictionary.load(os.path.join(TEMP_FOLDER, 'dictionary.dict'))
lsi = models.LsiModel.load(os.path.join(TEMP_FOLDER,'model.lsi'))
index=similarities.MatrixSimilarity.load(os.path.join(TEMP_FOLDER, 'index.index'))
answer_lines=[line for line in open(a_text)]

### Call the answer_to_user_query for each new query.

In [55]:
#Execute for each new query
answer_to_user_query(input("How can I help you? \n "), dictionary, lsi, index, answer_lines)

How can I help you? 
 Hot to add marker?
The sprint marker is a Scrum tool that can be used to quickly add multiple issues in the backlog into an inactive sprint....
Is this answer relevant?(yes/no)yes
Great! Here is the rest of the answer:
The sprint marker requires a Sprint to be created first before it becomes visible and ready to be used.
(1)In Plan Mode, create a sprint by clicking on the Create Sprint button.
(2)You should be able to see the Sprint marker below the inactive sprint.
(3)Simply click and drag the marker downwards to automatically include issues from the backlog into this sprint.



In [57]:
#Execute for each new query
answer_to_user_query(input("How can I help you? \n"), dictionary, lsi, index, answer_lines)
#Execute for each new query
answer_to_user_query(input("How can I help you? \n "), dictionary, lsi, index, answer_lines)

How can I help you? 
What is it?
Sorry, I do not know the answer.

How can I help you? 
 How can I set up authorization for scrum board?
Custom fields in JIRA (story points is a custom field) can be configured with a context that restricts it's usage to certain projects and/or issue types....
Is this answer relevant?(yes/no)no
Sorry. You will need to ask someone else
